In [1]:
from langchain_community.utilities.sql_database import SQLDatabase


In [2]:
# loading a database using langchain

db = SQLDatabase.from_uri("sqlite:///my_db.db")

In [3]:
db

In [4]:
db.dialect

'sqlite'

In [5]:
db.get_usable_table_names()

['customer', 'employee', 'orders']

In [6]:
pip install langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [7]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("API_KEY")

In [8]:
from langchain_groq import ChatGroq

In [9]:
llm = ChatGroq(model="llama3-70b-8192", api_key = api_key)

In [10]:
llm.invoke("hy how are you")

AIMessage(content='Hi! I\'m just an AI, I don\'t have feelings or emotions like humans do, so I don\'t have good or bad days. I\'m always "on" and ready to help with any questions or tasks you may have! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 14, 'total_tokens': 70, 'completion_time': 0.120875919, 'prompt_time': 0.010458939, 'queue_time': 0.267851318, 'total_time': 0.131334858}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b2de1b63-fea9-4724-8698-3c606d9955b0-0', usage_metadata={'input_tokens': 14, 'output_tokens': 56, 'total_tokens': 70})

In [11]:
# model is working fine as of now

# Langchain Tools 

In [20]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [21]:
toolkit = SQLDatabaseToolkit(db = db, llm = llm)

In [22]:
tools = toolkit.get_tools()

In [23]:
tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7400a7f4aa10>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7400a7f4aa10>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7400a7f4aa10>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [24]:
# checking what tools i will be getting in toolkit
for tool in tools:
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [25]:
# to get any tool 
sql_db_query = next((tool for tool in tools if tool.name == "sql_db_query"), None)

In [26]:
sql_db_query

QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7400a7f4aa10>)

In [27]:
list_table_tool = next((tool for tool in tools if tool.name == "sql_db_list_tables"), None)

In [30]:
list_table_tool.invoke("")

'customer, employee, orders'

In [31]:
schema_tool_db  =next(tool for tool in tools if tool.name == "sql_db_schema")

In [34]:
print(schema_tool_db.invoke("customer"))


CREATE TABLE customer (
	customer_id INTEGER, 
	first_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone TEXT, 
	PRIMARY KEY (customer_id), 
	UNIQUE (email)
)

/*
3 rows from customer table:
customer_id	first_name	email	phone
1	a	a@gmail.com	123456
2	b	b@gmail.com	234567
3	c	c@gmail.com	345678
*/


In [ ]:
# data base cannot be directly connected with langchain
# we have created database wrapup and will use this in a flow